In [1]:
import logging as log
import math
import itertools as it
import numpy as np
import scipy.special
import scipy.stats

import torch
import pandas as pd
import numpy as np
from pathlib import Path
from typing import *
%matplotlib inline
import sys
import os

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


WEAT data copying

In [ ]:
! cp -R /content/drive/MyDrive/Contextual_Bias_Data/weat_bn_data /content/

SEAT data copying

In [ ]:
! cp -R /content/drive/MyDrive/Contextual_Bias_Data/seat_bn_data /content/

GloVe model copying

In [ ]:
! cp -R /content/drive/MyDrive/Contextual_Bias_Data/bn_glove.39M.300d.txt /content/

word2vec model copying

In [ ]:
! cp -R /content/drive/MyDrive/Contextual_Bias_Data/bangla_embeddings/full_data/w2v_512 /content/

In [2]:
! pip install bnlp_toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.2 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=d79aacde52a3f4e28e43c4ac03febb61af26ca6d3cf7c22f4d82ef1d1d649146
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: tqdm
    Found exist

In [1]:
! pip install gensim

In [2]:
! pip install git+https://github.com/csebuetnlp/normalizer
! pip install transformers

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-zdubnmyl
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-zdubnmyl
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6860 sha256=7c0df5d7a5cf266fbf7eb42090286560a009faf852108e2125cff9b19d43e820
  Stored in directory: /tmp/pip-ephem-wheel-cache-tv7wg4sg/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186455 sha256=65f1d090beb71f78632ffeb28b20e54fb669e83

In [3]:
from bnlp import BengaliGlove

punkt not found. downloading...


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
glove_path = '/content/bn_glove.39M.300d.txt'

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
If you are using version <=3.3.2 please specify bnlp_toolkit with exact version, otherwise it will raises error in the upcoming version. 
To migrate feel free to checkout the newer version (4.0.0). It will release soon as beta.


In [4]:
from gensim.models import Word2Vec

### Model fetching from transformers

In [7]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from normalizer import normalize

In [6]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert_large")
model = AutoModelForMaskedLM.from_pretrained("csebuetnlp/banglabert_large", output_hidden_states = True)
model.eval()

tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at csebuetnlp/banglabert_large and are newly initialized: ['generator_lm_head.bias', 'generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight', 'generator_predictions.dense.bias', 'generator_predictions.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForMaskedLM(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-23): 24 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): Lay

### WEAT implementing

Json data loading

In [8]:
import json

In [9]:
def load_json(sent_file):
    print(f"Loading {sent_file}...")
    all_data = json.load(open(sent_file, 'r'))
    data = {}
    targets = [ all_data['targ1']['category'], all_data['targ2']['category'] ]
    attributes = [all_data['attr1']['category'], all_data['attr2']['category']]
    for k, v in all_data.items():
        examples = v["examples"]
        data[k] = examples
        v["examples"] = examples

    return all_data, targets, attributes

In [ ]:
data, targets, attributes = load_json('/content/weat_bn_data/weat6b.jsonl')
print(targets)
print(attributes)

In [ ]:
def encode(model, data, MODEL_NAME):
    dict_word2vec = {}
    if MODEL_NAME == 'glove':
        for word in data:
            vector = model.word2vec(glove_path, word)
            dict_word2vec[word] = vector
    elif MODEL_NAME == 'w2v':
        for word in data:
            vector = model.wv[word]
            dict_word2vec[word] = vector

    elif MODEL_NAME == 'bert':
        for i, word in enumerate(data['words']):
            itr = 0
            vector_sum = np.zeros(1024, dtype=float)
            while itr < data['count']:
                word_vector, _ = get_word_vector_normal(data['examples'][i*data['count'] + itr], word)
                vector_sum = np.add(vector_sum, word_vector)
                itr +=1
            average_vector = vector_sum / data['count']
            dict_word2vec[word] = average_vector

    return dict_word2vec

In [ ]:
dict_word2vec = encode(model, all_seat_data[0]['targ1'], 'bert')
type(dict_word2vec['গোলাপ'])

In [10]:
def encode_data(model, data, MODEL_NAME, suppress_printables = False):
    if suppress_printables == False:
        print('encoding data...')
    if MODEL_NAME == 'bert':
        encs_targ1 = encode(model, data["targ1"], MODEL_NAME)
        encs_targ2 = encode(model, data["targ2"], MODEL_NAME)
        encs_attr1 = encode(model, data["attr1"], MODEL_NAME)
        encs_attr2 = encode(model, data["attr2"], MODEL_NAME)
    else:
        encs_targ1 = encode(model, data["targ1"]["examples"], MODEL_NAME)
        encs_targ2 = encode(model, data["targ2"]["examples"], MODEL_NAME)
        encs_attr1 = encode(model, data["attr1"]["examples"], MODEL_NAME)
        encs_attr2 = encode(model, data["attr2"]["examples"], MODEL_NAME)

    return encs_targ1, encs_targ2, encs_attr1, encs_attr2

In [11]:
def add_encodings_to_dict(data, encs_targ1, encs_targ2, encs_attr1, encs_attr2, suppress_printables = False):
    if suppress_printables == False:
        print('adding encoded vectors to data dict...')
    data["targ1"]["encs"] = encs_targ1
    data["targ2"]["encs"] = encs_targ2
    data["attr1"]["encs"] = encs_attr1
    data["attr2"]["encs"] = encs_attr2

In [ ]:
CATEGORIES = ['targ1', 'targ2', 'attr1', 'attr2']
def save_encodings(data, filename):
    for category in CATEGORIES:
        for word, vec in data[category]['encs'].items():
            data[category]['encs'][word] = data[category]['encs'][word].tolist()
    with open('/content/weat_bn_encoded_data/weat8b_enc.jsonl', 'w') as f:
        json.dump(data, f)


Similarity score calculating

In [12]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [13]:
def construct_cossim_lookup(XY, AB):
    cossims = np.zeros((len(XY), len(AB)))
    for xy in XY:
        for ab in AB:
            cossims[xy, ab] = cosine_similarity(XY[xy], AB[ab])
    return cossims

In [14]:
def s_wAB(A, B, cossims):
    return cossims[:, A].mean(axis=1) - cossims[:, B].mean(axis=1)

In [15]:
def s_XAB(X, s_wAB_memo):
    return s_wAB_memo[X].sum()

def s_XYAB(X, Y, s_wAB_memo):
    return s_XAB(X, s_wAB_memo) - s_XAB(Y, s_wAB_memo)

Permutation testing

In [16]:
def p_val_permutation_test(X, Y, A, B, n_samples, cossims, parametric=False, suppress_printables = False):
    X = np.array(list(X), dtype=int)
    Y = np.array(list(Y), dtype=int)
    A = np.array(list(A), dtype=int)
    B = np.array(list(B), dtype=int)

    assert len(X) == len(Y)
    size = len(X)
    s_wAB_memo = s_wAB(A, B, cossims=cossims)
    XY = np.concatenate((X, Y))

    if parametric:
        if suppress_printables == False:
            print('Using parametric test')
        s = s_XYAB(X, Y, s_wAB_memo)

        if suppress_printables == False:
            print('Drawing {} samples'.format(n_samples))
        samples = []
        for _ in range(n_samples):
            np.random.shuffle(XY)
            Xi = XY[:size]
            Yi = XY[size:]
            assert len(Xi) == len(Yi)
            si = s_XYAB(Xi, Yi, s_wAB_memo)
            samples.append(si)

        if suppress_printables == False:
            print('Inferring p-value based on normal distribution')
        (shapiro_test_stat, shapiro_p_val) = scipy.stats.shapiro(samples)
        if suppress_printables == False:
            print('Shapiro-Wilk normality test statistic: {:.2g}, p-value: {:.2g}'.format(
            shapiro_test_stat, shapiro_p_val))
        sample_mean = np.mean(samples)
        sample_std = np.std(samples, ddof=1)
        if suppress_printables == False:
            print('Sample mean: {:.2g}, sample standard deviation: {:.2g}'.format(
            sample_mean, sample_std))
        p_val = scipy.stats.norm.sf(s, loc=sample_mean, scale=sample_std)
        return p_val

    else:
        if suppress_printables == False:
            print('Using non-parametric test')
        s = s_XAB(X, s_wAB_memo)
        total_true = 0
        total_equal = 0
        total = 0

        num_partitions = int(scipy.special.binom(2 * len(X), len(X)))
        if num_partitions > n_samples:
            total_true += 1
            total += 1
            if suppress_printables == False:
                print('Drawing {} samples (and biasing by 1)'.format(n_samples - total))
            for _ in range(n_samples - 1):
                np.random.shuffle(XY)
                Xi = XY[:size]
                assert 2 * len(Xi) == len(XY)
                si = s_XAB(Xi, s_wAB_memo)
                if si > s:
                    total_true += 1
                elif si == s:
                    total_true += 1
                    total_equal += 1
                total += 1

        else:
            if suppress_printables == False:
                print('Using exact test ({} partitions)'.format(num_partitions))
            for Xi in it.combinations(XY, len(X)):
                Xi = np.array(Xi, dtype=
                              int)
                assert 2 * len(Xi) == len(XY)
                si = s_XAB(Xi, s_wAB_memo)
                if si > s:
                    total_true += 1
                elif si == s:
                    total_true += 1
                    total_equal += 1
                total += 1

        if total_equal:
            if suppress_printables == False:
                print('Equalities contributed {}/{} to p-value'.format(total_equal, total))

        return total_true / total

Mean and Standard deviation calculating

In [17]:
def mean_s_wAB(X, A, B, cossims):
    return np.mean(s_wAB(A, B, cossims[X]))

def stdev_s_wAB(X, A, B, cossims):
    return np.std(s_wAB(A, B, cossims[X]), ddof=1)

Effect size calculating

In [18]:
def effect_size(X, Y, A, B, cossims):
    X = list(X)
    Y = list(Y)
    A = list(A)
    B = list(B)

    numerator = mean_s_wAB(X, A, B, cossims=cossims) - mean_s_wAB(Y, A, B, cossims=cossims)
    denominator = stdev_s_wAB(X + Y, A, B, cossims=cossims)
    return numerator / denominator

In [19]:
def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [20]:
def run_test(encs, n_samples, parametric=False, suppress_printables = False):
    X, Y = encs["targ1"]["encs"], encs["targ2"]["encs"]
    A, B = encs["attr1"]["encs"], encs["attr2"]["encs"]

    (X, Y) = convert_keys_to_ints(X, Y)
    (A, B) = convert_keys_to_ints(A, B)

    XY = X.copy()
    XY.update(Y)
    AB = A.copy()
    AB.update(B)

    if suppress_printables == False:
        print("Computing cosine similarities...")
    cossims = construct_cossim_lookup(XY, AB)

    if suppress_printables == False:
        print(f"Null hypothesis: no difference between {encs['targ1']['category']} and {encs['targ2']['category']} in \
    association to attributes {encs['attr1']['category']} and {encs['attr2']['category']}")
        print("Computing pval...")
    pval = p_val_permutation_test(X, Y, A, B, n_samples, cossims=cossims, parametric=parametric, suppress_printables=suppress_printables)
    print(f"pval: {pval}")

    if suppress_printables == False:
        print("computing effect size...")
    esize = effect_size(X, Y, A, B, cossims=cossims)
    print(f"Effect size: {esize}")
    return esize, pval

In [ ]:
X = {"x" + str(i): 2 * np.random.rand(10) - 1 for i in range(25)}
Y = {"y" + str(i): 2 * np.random.rand(10) - 1 for i in range(25)}
A = {"a" + str(i): 2 * np.random.rand(10) - 1 for i in range(25)}
B = {"b" + str(i): 2 * np.random.rand(10) - 1 for i in range(25)}
A = X
B = Y
print(f'X: {X}')
print(f'Y: {Y}')
(X, Y) = convert_keys_to_ints(X, Y)
print(f'X: {X}')
print(f'Y: {Y}')
(A, B) = convert_keys_to_ints(A, B)

XY = X.copy()
XY.update(Y)
AB = A.copy()
AB.update(B)

cossims = construct_cossim_lookup(XY, AB)
print("computing pval...")
pval = p_val_permutation_test(X, Y, A, B, cossims=cossims, n_samples=10000)
print("pval: %g", pval)

print("computing effect size...")
esize = effect_size(X, Y, A, B, cossims=cossims)
print(f"Effect size: {esize}")

Intended model loading

In [ ]:
MODEL_NAME = "glove"

if MODEL_NAME == 'w2v':
    print('Loading w2v model...')
    model = Word2Vec.load("/content/w2v_512/w2v_512")
elif MODEL_NAME == 'glove':
    print('Loading glove model...')
    model = BengaliGlove()

Loading glove model...


WEAT on single data

In [ ]:
filename = '/content/weat_bn_data/weat8b.jsonl'
data, targets, attributes = load_json(filename)

et1, et2, ea1, ea2 = encode_data(model, data, MODEL_NAME, suppress_printables = False)
add_encodings_to_dict(data, et1, et2, ea1, ea2, suppress_printables = False)
save_encodings(data, filename)
run_test(data, 10000, suppress_printables = False)

print(f"{targets[0]} vs {targets[1]} ({attributes[0]} / {attributes[1]})")

Saved encoding loading and GloVe test re-running

In [ ]:
data_enc = json.load(open('/content/weat_bn_encoded_data/weat8b_enc.jsonl', 'r'))
run_test(data, 10000, parametric = True, suppress_printables = False)

WEAT on whole dataset

In [ ]:
folder_path = '/content/weat_bn_data'
files = os.listdir(folder_path)

for file in files:
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path):
        data, targets, attributes = load_json(file_path)
        et1, et2, ea1, ea2 = encode_data(model, data, MODEL_NAME, suppress_printables = True)
        add_encodings_to_dict(data, et1, et2, ea1, ea2, suppress_printables = True)

        print(f"{targets[0]} vs {targets[1]} ({attributes[0]} / {attributes[1]})")
        run_test(data, 10000, suppress_printables = True)
        print()

### WEAT for BERT implementing

In [22]:
def get_word_vector_broken(sentence, word):
  normalized_sent = normalize(sentence)
  input_token = tokenizer(normalized_sent, return_tensors="pt")
  sent_list = sentence.split(' ')
  idx = sent_list.index(word) + 1
  with torch.no_grad():
    outputs = model(**input_token)
    print(outputs.hidden_states[-1][0][idx])
    return outputs[1][-1][0].detach().cpu().numpy()[idx]

In [ ]:
sentence = "ছেলেরা বিকেলে মাঠে ফুটবল খেলে।"
get_word_vector(sentence, 'খেলে')

get_word_vector testing

In [23]:
import re

In [24]:
def get_word_vector_normal(sentence, word):

    normalized_sent = normalize(sentence)
    word = normalize(word)
    input_token_mappings = tokenizer(normalized_sent, return_tensors="pt", return_offsets_mapping = True)
    input_token = tokenizer(normalized_sent, return_tensors="pt")
    decoded = tokenizer.decode(input_token['input_ids'][0])
    sent_list = normalized_sent.split(' ')
    if word in sent_list:
        idx = sent_list.index(word) + 1
    else:
        pattern = r'\b' + word + r'\W*'
        for i, w in enumerate(sent_list):
            if re.search(pattern, w):
                idx = i + 1
    with torch.no_grad():
        outputs = model(**input_token)
        return outputs.hidden_states[-1][0].detach().cpu().numpy()[idx], input_token_mappings

In [ ]:
sentence = "এটি একটি কৃষ্ণচূড়া।"
print(get_word_vector_normal(sentence, 'কৃষ্ণচূড়া'))

In [ ]:
sentence = "বাগানে রজনীগন্ধা ফুল ফুটেছে।"
sentence2 = "বাগানে গোলাপ ফুল ফুটেছে।"
vector1 = get_word_vector_broken(sentence, 'রজনীগন্ধা')
vector2, _ = get_word_vector_normal(sentence2, 'গোলাপ')
vector2.shape

tensor([ 0.1118,  0.3700,  0.7958,  ...,  0.4475,  1.1334, -0.2145])


(1024,)

In [25]:
import json

In [26]:
def load_json_templates(sent_file):
    print(f"Loading {sent_file}...")
    all_data = json.load(open(sent_file, 'r'))
    data = {}
    targets = [ all_data['targ1']['category'], all_data['targ2']['category'] ]
    attributes = [all_data['attr1']['category'], all_data['attr2']['category']]
    for k, v in all_data.items():
        templates = v["templates"]
        data[k] = templates
        v["templates"] = templates

    return all_data, targets, attributes

In [ ]:
folder_path = '/content/seat_bn_data/templates'
files = os.listdir(folder_path)
all_seat_template_data = []
files.sort()
for file in files:
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path):
        seat_template_data, targets, attributes = load_json_templates(file_path)
        all_seat_template_data.append(seat_template_data)
        print(f"{seat_template_data['targ1']['category']} has {len(seat_template_data['targ1']['templates'])} sentences")
        print(f"{seat_template_data['targ2']['category']} has {len(seat_template_data['targ2']['templates'])} sentences")
        print(f"{seat_template_data['attr1']['category']} has {len(seat_template_data['attr1']['templates'])} sentences")
        print(f"{seat_template_data['attr2']['category']} has {len(seat_template_data['attr2']['templates'])} sentences")

In [ ]:
weat_folder_path = '/content/weat_bn_data'
weat_files = os.listdir(weat_folder_path)
weat_files.sort()
all_weat_data = []

for file in weat_files:
    weat_file_path = os.path.join(weat_folder_path, file)
    if os.path.isfile(weat_file_path):
        weat_data, targets, attributes = load_json(weat_file_path)
        all_weat_data.append(weat_data)
        print(f"{weat_data['targ1']['category']} has {len(weat_data['targ1']['examples'])} words")
        print(f"{weat_data['targ2']['category']} has {len(weat_data['targ2']['examples'])} words")
        print(f"{weat_data['attr1']['category']} has {len(weat_data['attr1']['examples'])} words")
        print(f"{weat_data['attr2']['category']} has {len(weat_data['attr2']['examples'])} words")

In [ ]:
for i, data in enumerate(all_weat_data):
    data['targ1']['count'] = len(all_seat_template_data[i]['targ1']['templates'])
    data['targ2']['count'] = len(all_seat_template_data[i]['targ2']['templates'])
    data['attr1']['count'] = len(all_seat_template_data[i]['attr1']['templates'])
    data['attr2']['count'] = len(all_seat_template_data[i]['attr2']['templates'])

In [ ]:
all_seat_data[0]

In [ ]:
folder_path = '/content/seat_bn_data/data'
files = os.listdir(folder_path)
all_seat_data = []
files.sort()
for file in files:
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path):
        seat_data, targets, attributes = load_json(file_path)
        all_seat_data.append(seat_data)
        print(f"{seat_data['targ1']['category']} has {len(seat_data['targ1']['examples'])} sentences")
        print(f"{seat_data['targ2']['category']} has {len(seat_data['targ2']['examples'])} sentences")
        print(f"{seat_data['attr1']['category']} has {len(seat_data['attr1']['examples'])} sentences")
        print(f"{seat_data['attr2']['category']} has {len(seat_data['attr2']['examples'])} sentences")

In [ ]:
for i, seat_data in enumerate(all_seat_data):
    seat_data['targ1']['words'] = all_weat_data[i]['targ1']['examples']
    seat_data['targ1']['count'] = all_weat_data[i]['targ1']['count']

    seat_data['targ2']['words'] = all_weat_data[i]['targ2']['examples']
    seat_data['targ2']['count'] = all_weat_data[i]['targ2']['count']

    seat_data['attr1']['words'] = all_weat_data[i]['attr1']['examples']
    seat_data['attr1']['count'] = all_weat_data[i]['attr1']['count']

    seat_data['attr2']['words'] = all_weat_data[i]['attr2']['examples']
    seat_data['attr2']['count'] = all_weat_data[i]['attr2']['count']

Sanity checking

In [ ]:
for i, seat_data in enumerate(all_seat_data):
    if len(seat_data['targ1']['words'])*seat_data['targ1']['count'] != len(seat_data['targ1']['examples']):
        print("Issue")
    if len(seat_data['targ2']['words'])*seat_data['targ2']['count'] != len(seat_data['targ2']['examples']):
        print("Issue")
    if len(seat_data['attr1']['words'])*seat_data['attr1']['count'] != len(seat_data['attr1']['examples']):
        print("Issue")
    if len(seat_data['attr2']['words'])*seat_data['attr2']['count'] != len(seat_data['attr2']['examples']):
        print("Issue")

SEAT on single file

In [ ]:
MODEL_NAME = 'bert'

et1, et2, ea1, ea2 = encode_data(model, all_seat_data[0], MODEL_NAME, suppress_printables = False)
add_encodings_to_dict(all_seat_data[0], et1, et2, ea1, ea2, suppress_printables = False)
run_test(all_seat_data[0], 10000, suppress_printables = False)

SEAT on whole dataset

In [ ]:
MODEL_NAME = 'bert'
for seat_data in all_seat_data:
    et1, et2, ea1, ea2 = encode_data(model, seat_data, MODEL_NAME, suppress_printables = False)
    add_encodings_to_dict(seat_data, et1, et2, ea1, ea2, suppress_printables = False)
    run_test(seat_data, 10000, suppress_printables = False)
    print(f"{seat_data['targ1']['category']} vs {seat_data['targ2']['category']} ({seat_data['attr1']['category']} / {seat_data['attr2']['category']})")